## Compute Consensus Scoring using Protein Conformational Selection

In [1]:
import pandas as pd
import numpy as np
import glob, os, sys
sys.path.append('..')
from modules.run_or_load_decorator import run_or_load

In [2]:
# Function to compute Consensus scoring giving a set of selected conformations

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
def format_name(*args, sep='/'):
    return sep.join([*args])

In [5]:
def evaluate_consensus_scoring(
                              X, y,
                              cs_method,
                              test_size=0.25,
                              metric_params=None
                             ):
    '''
        Applies a consensus scoring method (cs_method) over a set (X) of dksc scores
        and evaluates its performance using a evaluation metric (metric_params).
        Returns a dataframe with the metric score value.
    '''
    consensus_score=cs_method(X)
    
    dict_y_preds={'y_pred': consensus_score}
    
    metric_results=PlotMetric(y_true=y,  y_pred_dict=dict_y_preds, 
           decreasing=False).format_metric_results(**metric_params)
    
    return(metric_results.T)

In [6]:
def split_Xy_for_CS(
                    X, y, 
                    selector, 
                    splitting_method, 
                    test_size=0.25,
                    scaffold_series=None
                    ):
    '''
        Splits X and y using a specific train-test splitting method
    ''' 

    try: 
        scaffold_series
    except NameError:
        print("scaffold_series object doesn't exist")

    n = X.shape[1]
    
    # Initialize X_test and y_test as X and y
    X_test, y_test = None, None
    # If 'none' then do no perform a split and assume the whole set as test set
    if splitting_method == 'none':
        X_test, y_test = X, y
    elif splitting_method == 'rand':
        # Perform a random splitting
        _, X_test, _, y_test = train_test_split(X, y, 
                                             test_size=test_size, stratify=y)
    elif splitting_method == 'scff':
        
        _, X_test, _, y_test = train_test_scaffold_split(X, y, 
                                             scaffold_series=scaffold_series,
                                             test_size=test_size, stratify=y)
    return X_test, y_test 

In [7]:
def get_CS_score_in_n_confs_range(X, y, splitting_method, selector,
                                 cs_method, test_size, metric_params,
                                 scaffold_series=None):
    
    # First verify that rfe_preselections object is in memory
    try:
       rfe_preselections 
    except NameError:
        print("rfe_preselections object doesn't exist")
        
    # Get the total number of selections
    n_confs = X.shape[1]
    # Initialize X_test and y_test 
    X_test, y_test = None, None
    confs = None
    # List of scoring results
    results_list = []
    
    # Evaluate the Consensus method in range of 1 to n conformations
    for k in range(1, n_confs + 1):
        # *************
        # Splitting
        # *************
        if splitting_method == 'rand':
        # If is random, repeat splitting at each k step
            X_test, y_test = split_Xy_for_CS(X, y, selector=selector, 
                                           splitting_method=splitting_method, test_size=test_size)
        # If splitting method is not random, just do splittin once
        elif k == 1:
            X_test, y_test = split_Xy_for_CS(X, y, selector=selector, 
                                           splitting_method=splitting_method, test_size=test_size,
                                           scaffold_series=scaffold_series)
        
        # ***********************
        # Conformation Selection 
        # ***********************
        if selector == 'rand':
            # make a random sampling without replacement and select the first k positions
            confs = np.random.choice(a=range(n_confs), size=n_confs, replace=False)[:k]
        else:
            confs = rfe_preselections[selector + '_' + splitting_method][:k] 
            
        # Subset the X_test
        X_test_confs = X_test.iloc[:, confs]
        
        # ****************
        # Evaluation Score
        # ****************
        score = evaluate_consensus_scoring(
                          X=X_test_confs, y=y_test, 
                          cs_method=cs_method, 
                          test_size=test_size,
                          metric_params=metric_params)
    
        results_list.append(score.values[0][0])
        
    return np.array(results_list)

In [8]:
def do_CS_replicas(
        nreps=15, 
        **kwargs
    ):
    
    results_dict = {}
    
    for rep in range(nreps):
        scores_list = get_CS_score_in_n_confs_range(**kwargs)
        
        results_dict[rep] = scores_list
        
    df = pd.DataFrame(results_dict)
    # Get the mean and standard deviation
    df = df.apply([np.mean, np.std], axis=1).fillna(0)
    return df 

In [9]:
@run_or_load
def aggregate_conf_selection_results_CS(
        filename,
        X,
        y,
        splitting_methods,
        selectors,
        cs_methods,
        metrics,
        test_size=0.25,
        nreps=15,
        scaffold_series=None
    ):
    
    #*********************************************************
    # Compute Results from Consensus Scoring over 1 to N confs
    #*********************************************************
    results_cs = []
    for metric in metrics:
        for split in splitting_methods:
            for selector in selectors:
                # Skip if RFE slector and none splitting
                if split == 'none' and selector != 'rand':
                     break
                for cs_name, csm in cs_methods.items():
                    metric_name = '_'.join([str(i) for i in metric.values()])
                    print(format_name(split, selector, cs_name, metric_name)) 
                    df = do_CS_replicas(nreps=nreps,
                                   X=X, y=y, 
                                   splitting_method=split,
                                   selector=selector,
                                   cs_method=csm, 
                                   test_size=test_size,
                                   metric_params=metric,
                                   scaffold_series=scaffold_series)
                    # Format colnames
                    df.columns = (format_name(split, selector, cs_name, metric_name, 'mean'),
                                  format_name(split, selector, cs_name, metric_name, 'std'))

                    results_cs.append(df.T)
    X_cs = pd.concat(results_cs)
    X_cs.index = X_cs.index.str.split('/', expand=True)
    X_cs.rename_axis(("split", "selector", "consensus", "metric", "desc"), inplace=True)
    
    return X_cs 

# Consensus Scoring

### *Rank by number*
<div style='background-color: #F9E5AB; min-height: 10px'></div>

> The final score of each ligand $i$ ($R_bN_i$) corresponds to the *mean score* of ligand $i$ from a set of $n$ conformations.

> $R_bN_i = \frac{1}{n}\sum_js_i^j,$

where $n$ is the number of conformations, and $s_i^j$ is the docking score of the molecule $i$ with the $j$ conformation.

In [10]:
def get_rank_by_number(df):
    '''Get a dataframe of m*n values and returns a numpy array of means per row'''
    # times -1 because roc_auc_score assumes higher is better
    df_means = df.mean(axis = 1).to_numpy() * -1
    return df_means

### *Best score (Merging and shrinking strategy)*
<div style='background-color: #F9E5AB; min-height: 10px'></div>

Also mentioned as *Merging and shrinking strategy* by Palacio-Rodríguez, et al. (2019).

For each molecule $i$, get the lowest score of the $n$ scores calculated.
> $
\begin{aligned}
best\ score_i = min(s_i),
\end{aligned}
$

where $s_i$ is a vector of $n$ scores, belonging to the molecule $i$, in which each position $j$ corresponds to the docking score between the molecule $i$ and each of the $n$ protein conformations.

In [11]:
def get_best_score(df):
    '''Get a dataframe of m*n values and returns a numpy array of best scores per row'''
    df_best = df.min(axis = 1)
    # (times -1 because roc_auc_score in the following line assumes higher is better)
    return df_best.to_numpy() * -1

### *Rank-by-rank*
<div style='background-color: #F9E5AB; min-height: 10px'></div>

For each conformation $j$ the $m$ molecules are ranked according to their docking score with that conformation. Therefore, the molecule with the best score is assigned with the value 1, the second one with the best score is assigned with the value 2, and so on, until the molecule with the worst value is assigned with the value $m$. Finally, the average rank of each molecule is compute taking into account its rank in all the $n$ conformations.

> $
\begin{aligned}
RbR_i = \frac{1}{n}\sum_jr^i_j,
\end{aligned}
$

where $n$ is the number of protein conformations, and $r^i_j$ is the rank position of the molecule $i$ in the ranked score obtained in the protein conformation $j$.

In [12]:
def get_rank_by_rank(df):
    df_ranks = df.rank() # First, we get the ranking positions
    # For each ligand i we get the average score among the n conformations
    df_rank_by_rank = df_ranks.mean(axis = 1)
    return df_rank_by_rank.to_numpy() * -1

### *Rank-by-vote*
<div style='background-color: #F9E5AB; min-height: 10px'></div>
Each molecule $i$ receives a vote if it is ranked on top $x\%$ of the ranking list of each conformation. The final score of the molecule is given by  the sum of votes obtained in all the protein conformations.

> $RbV_i(\chi) = \sum_j\delta^i_j,$

having:
> - $\delta _i = 1$ if $r^i_j \leq \chi m$
- $\delta _i = 0$ if $r^i_j > \chi m$  

where $\chi$ is the top fraction of the docking scores ranking of $m$ molecules, $r^i$ is the rank of molecule $i$ in the ranking of the conformation $j$, and $n$ is the number of conformations.

In [13]:
def get_rank_by_vote(df, chi = 10):
    df_ranks = df.rank(ascending=True) # Get ranks, ascending=True; lower values get lower positions
    x = np.floor(df.shape[0]/100 * chi) # Get the number of molecules inside chi
    rbv_array = df_ranks.to_numpy() # Convert to a numpy array
    # So, find values above chi (x) and assign the 0 or 1
    rbv_array = np.where(rbv_array > x, 0, 1)
    # Get the sum of votes
    rb_votes = rbv_array.sum(axis = 1)
    return rb_votes

### *Exponential Consensus Ranking*
<div style='background-color: #F9E5AB; min-height: 10px'></div>

It follows the same idea than the RIE and BEDROC metrics, taking into account the position of each molecule inside the ranked results following a exponential distribution. It is proposed by Palacio-Rodríguez, et al. (2019) as a scoring consensus using different docking programs:

> "*Exponential Consensus Ranking (ECR), a strategy to combine the results from
several scoring functions/docking programs using an exponential distribution for each individual rank.*"

For each molecule $i$ at the protein conformation $j$ the *exponential score* $p(r^j_i)$ is calculated as follows:

> $$p(r^j_i) = \frac{1}{\sigma}exp\left( - \frac{r_i^j}{\sigma} \right),$$

where $p(r^j_i)$ is the rank of the molecule $i$ in the docking scoring results of the protein conformation $j$, and $\sigma$ is the expected value of the exponential distribution. Just to remember, the exponential distribution (in the form $f(x) = \lambda e^{(- \lambda x)}$) has a mean and an standard deviation of $1/\lambda$ which here in $ECR$ is referred as $\sigma$: ($\sigma = 1 / \lambda$). As was said in a previous notebook for RIE and BEDROC metrics, the parameter $\sigma$ can be understood as a fraction of the ranked list where the weight is important.
As said by Palacio-Rodríguez, et al.:
> "*This parameter $(\sigma)$ establishes the number of molecules for each scoring function that will be considered, i.e., the threshold of the dataset to be taken into account for the consensus. *"

Finally, the final $ECR$ score $P(i)$ of each molecule $i$ is computed as follows:

> $$P(i) = \sum_{j=1}^n p(r^j_i) = \frac{1}{\sigma} \sum_{j=1}^n exp\left( - \frac{r_i^j}{\sigma} \right)$$

In [14]:
def get_exp_consensus_ranking(df, sigma = 2):
    # ascending=True; lower values get lower positions
    ranks_array = df.rank(ascending=True).to_numpy()
    # Apply the exponential function
    exp_ranks = (1/ sigma)*(np.exp( - ranks_array/sigma))
    
    # For each ligand i we get the average score among the n conformations
    ecs = exp_ranks.sum(axis = 1)
    return ecs